In [40]:
import os

# Create folders
folders = ['data', 'notebooks', 'src', 'docs', 'outputs']
for folder in folders:
    os.makedirs(folder, exist_ok=True)

# Create files
files = {
    'README.md': '# SecuLog Insights\n\nProject description...',
    'requirements.txt': 'pandas>=1.5.0\nnumpy>=1.24.0\nmatplotlib>=3.7.0',
    '.gitignore': '__pycache__/\n*.pyc\n.ipynb_checkpoints/',
    'src/__init__.py': '# Security log analysis package',
    'data/sample_logs.csv': '''timestamp,source_ip,destination_ip,port,action
2024-05-01 09:15:23,192.168.1.105,10.0.0.3,443,ALLOW
2024-05-01 09:15:24,192.168.1.105,10.0.0.3,443,ALLOW
2024-05-01 09:15:25,10.0.0.3,192.168.1.105,443,ALLOW
2024-05-01 09:16:01,203.0.113.17,192.168.1.105,22,DENY
2024-05-01 09:16:02,203.0.113.17,192.168.1.105,22,DENY
2024-05-01 09:16:03,203.0.113.17,192.168.1.105,22,DENY
2024-05-01 09:16:04,203.0.113.17,192.168.1.105,22,DENY
2024-05-01 09:16:05,203.0.113.17,192.168.1.105,22,DENY'''
}

for filepath, content in files.items():
    with open(filepath, 'w') as f:
        f.write(content)

print("Project structure created!")

Project structure created!


In [48]:
# Create all files one by one - SIMPLER VERSION

# 1. Create data_cleaner.py
data_cleaner_code = '''
import pandas as pd

def clean_security_logs(filepath):
    """Clean security log files"""
    try:
        df = pd.read_csv(filepath)
        
        # Fix combined headers issue
        if len(df.columns) == 1 and "," in str(df.iloc[0, 0]):
            header = df.iloc[0, 0]
            columns = [col.strip() for col in header.split(",")]
            
            data_rows = []
            for i in range(1, len(df)):
                if pd.notna(df.iloc[i, 0]):
                    row = str(df.iloc[i, 0]).split(",")
                    if len(row) == len(columns):
                        data_rows.append(row)
            
            df = pd.DataFrame(data_rows, columns=columns)
        
        # Clean column names
        df.columns = df.columns.str.strip()
        
        # Convert port to number if it exists
        if "port" in df.columns:
            df["port"] = pd.to_numeric(df["port"], errors="coerce")
            
        return df
        
    except Exception as e:
        print(f"Error: {e}")
        return None
'''

with open("src/data_cleaner.py", "w") as f:
    f.write(data_cleaner_code)

print("Created data_cleaner.py")

Created data_cleaner.py


In [49]:
# 2. Create threat_analyzer.py
threat_analyzer_code = '''
import pandas as pd

class ThreatAnalyzer:
    def __init__(self, log_data):
        self.df = log_data.copy()
    
    def analyze(self):
        summary = self._get_summary()
        attacks = self._detect_attacks()
        recommendations = self._get_recommendations(summary, attacks)
        
        return {
            "summary": summary,
            "attacks": attacks,
            "recommendations": recommendations
        }
    
    def _get_summary(self):
        total = len(self.df)
        attack_count = len(self.df[self.df["action"] == "DENY"])
        
        return {
            "total_connections": total,
            "attack_count": attack_count,
            "attack_percentage": (attack_count / total * 100) if total > 0 else 0
        }
    
    def _detect_attacks(self):
        attacks = self.df[self.df["action"] == "DENY"]
        
        if len(attacks) == 0:
            return {"status": "clean", "message": "No attacks"}
        
        attacker_ip = attacks.iloc[0]["source_ip"]
        
        return {
            "status": "compromised",
            "attacker_ip": attacker_ip,
            "attack_count": len(attacks),
            "target_port": attacks.iloc[0]["port"]
        }
    
    def _get_recommendations(self, summary, attacks):
        recs = []
        
        if summary["attack_percentage"] > 10:
            recs.append(f"High attack rate: {summary['attack_percentage']:.1f}%")
        
        if attacks["status"] == "compromised":
            recs.append(f"Block IP: {attacks['attacker_ip']}")
        
        if not recs:
            recs.append("Security status: Normal")
        
        return recs
'''

with open("src/threat_analyzer.py", "w") as f:
    f.write(threat_analyzer_code)

print("Created threat_analyzer.py")

Created threat_analyzer.py


In [50]:
# 3. Create demo.py
demo_code = '''
import sys
sys.path.append("src")

from data_cleaner import clean_security_logs
from threat_analyzer import ThreatAnalyzer

print("=" * 50)
print("SECULOG INSIGHTS - DEMONSTRATION")
print("=" * 50)

# Load and clean data
df = clean_security_logs("data/sample_logs.csv")
print(f"Loaded {len(df)} log entries")

# Analyze
analyzer = ThreatAnalyzer(df)
results = analyzer.analyze()

# Display results
summary = results["summary"]
attacks = results["attacks"]

print(f"\\nSECURITY SUMMARY:")
print(f"Total connections: {summary['total_connections']}")
print(f"Attack attempts: {summary['attack_count']}")
print(f"Attack percentage: {summary['attack_percentage']:.1f}%")

if attacks["status"] == "compromised":
    print(f"\\nATTACK DETECTED:")
    print(f"Attacker: {attacks['attacker_ip']}")
    print(f"Attempts: {attacks['attack_count']}")
    print(f"Target: Port {attacks['target_port']}")

print(f"\\nRECOMMENDATIONS:")
for rec in results["recommendations"]:
    print(f"• {rec}")

print("\\n" + "=" * 50)
print("ANALYSIS COMPLETE")
print("=" * 50)
'''

with open("demo.py", "w") as f:
    f.write(demo_code)

print("Created demo.py")

Created demo.py


In [51]:
# 4. Create README.md - using a list of lines
readme_lines = [
    "#SecuLog Insights",
    "",
    "**Automated Security Log Analysis Platform**",
    "",
    "## Overview",
    "This project analyzes security logs to detect threats and generate business insights.",
    "",
    "## Quick Start",
    "```bash",
    "python demo.py",
    "```",
    "",
    "## Sample Output",
    "```",
    "Total connections: 8",
    "Attack attempts: 5",
    "Attack percentage: 62.5%",
    "Attacker: 203.0.113.17",
    "```",
    "",
    "## Author",
    "Your Name - Cybersecurity & Data Analytics",
    ""
]

with open("README.md", "w") as f:
    f.write("\n".join(readme_lines))

print("Created README.md")

Created README.md


In [52]:
# 5. Create other files

# .gitignore
gitignore_content = """__pycache__/
*.pyc
.ipynb_checkpoints
.DS_Store
outputs/
"""

with open(".gitignore", "w") as f:
    f.write(gitignore_content)

# requirements.txt
with open("requirements.txt", "w") as f:
    f.write("pandas>=1.5.0")

# LICENSE (simple)
with open("LICENSE", "w") as f:
    f.write("MIT License")

# __init__.py
with open("src/__init__.py", "w") as f:
    f.write("# SecuLog Insights package")

print("Created .gitignore, requirements.txt, LICENSE, and __init__.py")

Created .gitignore, requirements.txt, LICENSE, and __init__.py


In [53]:
# 6. Test the project
print("Testing the project...")
print("-" * 30)

# Test imports
try:
    import sys
    sys.path.append("src")
    
    from data_cleaner import clean_security_logs
    from threat_analyzer import ThreatAnalyzer
    
    print("Imports successful")
    
    # Test data loading
    df = clean_security_logs("data/sample_logs.csv")
    print(f"Data loaded: {len(df)} rows")
    
    # Test analysis
    analyzer = ThreatAnalyzer(df)
    results = analyzer.analyze()
    
    print(f"Analysis complete")
    print(f"Attack %: {results['summary']['attack_percentage']:.1f}%")
    print(f"Attacker: {results['attacks']['attacker_ip']}")
    
except Exception as e:
    print(f"Error: {e}")

print("-" * 30)
print("Project is working!")

Testing the project...
------------------------------
Imports successful
Data loaded: 8 rows
Analysis complete
Attack %: 62.5%
Attacker: 203.0.113.17
------------------------------
Project is working!


In [54]:
# 7. First, save this notebook to the notebooks folder
print("Save your current notebook:")
print("1. Click 'File' → 'Save As'")
print("2. Navigate to 'notebooks' folder")
print("3. Save as: '01_security_analysis.ipynb'")
print("\\nThis will become part of your portfolio!")

Save your current notebook:
1. Click 'File' → 'Save As'
2. Navigate to 'notebooks' folder
3. Save as: '01_security_analysis.ipynb'
\nThis will become part of your portfolio!


In [56]:
# Re-create demo.py with proper encoding and without special characters
demo_code = '''#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
SecuLog Insights - Demonstration Script
"""

import sys
import os

# Add src to path
sys.path.append("src")

from data_cleaner import clean_security_logs
from threat_analyzer import ThreatAnalyzer

def main():
    print("=" * 50)
    print("SECULOG INSIGHTS - SECURITY ANALYSIS DEMO")
    print("=" * 50)
    
    # 1. Load and clean data
    print("\\nSTEP 1: Loading and cleaning data...")
    df = clean_security_logs("data/sample_logs.csv")
    print(f"[OK] Loaded {len(df)} log entries")
    print(f"[OK] Columns: {', '.join(df.columns)}")
    
    # 2. Analyze threats
    print("\\nSTEP 2: Analyzing threats...")
    analyzer = ThreatAnalyzer(df)
    results = analyzer.analyze()
    
    # 3. Display results
    print("\\nSTEP 3: Security Assessment Results")
    print("-" * 40)
    
    summary = results["summary"]
    print(f"Total connections: {summary['total_connections']}")
    print(f"Attack attempts: {summary['attack_count']}")
    print(f"Attack percentage: {summary['attack_percentage']:.1f}%")
    
    attacks = results["attacks"]
    if attacks["status"] == "compromised":
        print(f"\\n[ALERT] ATTACK DETECTED!")
        print(f"Attacker IP: {attacks['attacker_ip']}")
        print(f"Attack count: {attacks['attack_count']}")
        print(f"Target port: {attacks['target_port']}")
    
    # 4. Recommendations
    print("\\nSTEP 4: Security Recommendations")
    print("-" * 40)
    for rec in results["recommendations"]:
        print(f"* {rec}")
    
    print("\\n" + "=" * 50)
    print("[SUCCESS] ANALYSIS COMPLETE")
    print("=" * 50)

if __name__ == "__main__":
    main()
'''

# Write with UTF-8 encoding
with open("demo.py", "w", encoding="utf-8") as f:
    f.write(demo_code)

print("Re-created demo.py with proper UTF-8 encoding")

Re-created demo.py with proper UTF-8 encoding


In [61]:
# Create README.md line by line
readme_lines = [
    "# SecuLog Insights",
    "",
    "Automated Security Log Analysis Platform",
    "",
    "## Overview",
    "This project analyzes security logs to detect threats and generate business insights.",
    "",
    "## Quick Start",
    "```bash",
    "python demo.py",
    "```",
    "",
    "## Project Structure",
    "- data/ - Sample datasets",
    "- notebooks/ - Jupyter notebooks",
    "- src/ - Python modules",
    "- demo.py - Main script",
    "",
    "## Author",
    "Bamidele Adedeji",
    ""
]

with open('README.md', 'w') as f:
    f.write('\n'.join(readme_lines))

print("Created README.md")

Created README.md


In [60]:
# Run the demo script
!python demo.py


SECULOG INSIGHTS - SECURITY ANALYSIS DEMO

STEP 1: Loading and cleaning data...
[OK] Loaded 8 log entries
[OK] Columns: timestamp, source_ip, destination_ip, port, action

STEP 2: Analyzing threats...

STEP 3: Security Assessment Results
----------------------------------------
Total connections: 8
Attack attempts: 5
Attack percentage: 62.5%

[ALERT] ATTACK DETECTED!
Attacker IP: 203.0.113.17
Attack count: 5
Target port: 22

STEP 4: Security Recommendations
----------------------------------------
* High attack rate: 62.5%
* Block IP: 203.0.113.17

[SUCCESS] ANALYSIS COMPLETE


In [63]:
# Create a simple visualization script
viz_content = """#!/usr/bin/env python
# Simple visualization for attack patterns

import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append('src')

from data_cleaner import clean_security_logs

def create_attack_visualization():
    # Load data
    df = clean_security_logs('data/sample_logs.csv')
    
    # Calculate attack stats
    total = len(df)
    attacks = len(df[df['action'] == 'DENY'])
    legitimate = total - attacks
    
    # Create pie chart
    labels = ['Legitimate Traffic', 'Attack Traffic']
    sizes = [legitimate, attacks]
    colors = ['lightgreen', 'red']
    
    plt.figure(figsize=(8, 6))
    plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
    plt.title('Network Traffic Analysis: Attacks vs Legitimate')
    plt.axis('equal')
    
    # Save to outputs folder
    import os
    os.makedirs('outputs', exist_ok=True)
    plt.savefig('outputs/traffic_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"Visualization saved to outputs/traffic_analysis.png")
    print(f"Stats: {attacks}/{total} attacks ({attacks/total*100:.1f}%)")

if __name__ == "__main__":
    create_attack_visualization()
"""

with open('visualize.py', 'w', encoding='utf-8') as f:
    f.write(viz_content)

print("Created visualize.py for generating charts")

Created visualize.py for generating charts
